In [ ]:
def get_profile_url(text: str) -> str:
    "Searches for LinkedIn profile Pages"
    pass

In [1]:
import os

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.agents import Tool, create_react_agent, AgentExecutor



def lookup(name: str) -> str:

    template = """Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question:given the full name {input} I want you to get it me a link to their LinkedIn profile page.
    your answer should contain only a URL
    Thought:{agent_scratchpad}
    """

    tools_for_agent = []

    llm = ''

    prompt_template = PromptTemplate(template=template, input_variables=["input"])

    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=prompt_template)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools_for_agent,
        return_intermediate_steps=True,
        verbose=True,
        handle_parsing_errors=True,
    )
    linkedin_profile_url = agent_executor.invoke({"input": name})

    for k, v in linkedin_profile_url.items():
        if k == "output":
            return v

    # return 'https://www.linkedin.com/in/satyanadella'


Hello


In [4]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [7]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
import os
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

template = """You are an AI assistant designed to classify user queries. Your task is to determine whether a given query is a normal conversation or a ServiceDesk conversation. 
    A normal conversation includes everyday, casual, or non-service-related topics. Examples include discussing hobbies, plans, general inquiries, and personal matters.
    A ServiceDesk conversation involves requests for assistance, support issues, troubleshooting, or inquiries related to IT services, technical support, or customer service.
    For each query, respond with either "Conversation" if it is a normal conversation or "ServiceDesk" if it is a ServiceDesk conversation.
    
    Return the classfication a JSON with a single key 'result' and no premable or explaination.

    Query: {query}
    Classification: 
    """

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

name = "what is mac new version?"

prompt_template = PromptTemplate(template=template, input_variables=["query"])
chain = prompt_template | model | JsonOutputParser()
response = chain.invoke({"query": name})
print(response)


{'result': 'ServiceDesk'}
